In [4]:
import json
import requests
import csv
import jsonlines
from flask import Flask, redirect, render_template, request, url_for


In [2]:
def save_json(data, filepath=r'new_data.json'):
    with open(filepath, 'w') as fp:
        json.dump(data, fp, indent=4)

In [75]:
def request_chatgpt(prompt):
    # input_filepath = r'./data/tweets/{}/'.format(case_name)
    original_url = "http://127.0.0.1:5000/event_extraction"
    body = {"prompt": prompt}
    response = requests.post(original_url, json=body).json()
    gpt_response = response['choices'][0]['text'].strip()
    return gpt_response

In [5]:
dev_reader = jsonlines.open(r'../data/raw/RAMS/dev.jsonlines')
dataset = [datum for datum in dev_reader]

In [6]:
def merge_sentences(datum_sentences):
    sentence_list = [" ".join(sentence_word_list) for sentence_word_list in datum_sentences] # merge the words into sentences
    paragraph = " ".join(sentence_list)
    return paragraph


In [66]:
article = merge_sentences(dataset[4]['sentences'])
article

'His campaign has just announced it will soon launch a new website , LyingCrookedHillary.com . Here ’s are just a few of the most egregious examples of Mrs. Clinton ’s corruption scandals : The commodities trader who “ bought ” his appointment to a Top Secret nuclear weapons security advisory board . Mr. Trump began speaking about commodities trader Raj Fernando on Friday , just hours after ABC News chief investigative correspondent Brian Ross broke the story that tied Mr. Fernando ’s appointment to the prestigious International Security Advisory Board in 2009 to his donations to the Clinton Foundation . Newly released emails , acquired by Citizens United through the Freedom of Information Act , show that career Department officials questioned Fernando ’s appointment because he had no national security experience .'

In [71]:
def get_arguments(article):
    prompt = """
    Below is a news article of an event.
    Please describe the main characters that the news article discussed, the character can be any organization, person or location.
    It can have one or more characters.
    Reply in the format '[character 1] [character 2]...'
    Article: \n {article}
    """.format(article=article)
    arguments = request_chatgpt(prompt)
    return arguments


In [73]:
def summarize_sentence(article, arguments):
    prompt = """
    Below is a news article of an event.
    The major participants in the articles are: {participants}.
    Please describe what the article discussed about them in one sentence.
    Reply starts with 'The article discussed ...'
    Article: \n {article}
    """.format(participants=", ".join(arguments), article=article)
    sentence = request_chatgpt(prompt)
    return sentence


In [77]:
saved_dataset = []
for datum in dataset:
    saved_datum = {}
    article = merge_sentences(datum['sentences'])
    arguments = get_arguments(article)
    sentence = summarize_sentence(article, arguments)
    print(sentence)
    saved_datum['content'] = datum['sentences']
    saved_datum['url'] = datum['source_url']
    saved_datum['summary'] = sentence
    saved_dataset.append(saved_datum)
save_json(saved_dataset, r'../data/raw/RAMS/summarized/dev.json')

The article discussed how Trump's inability to work with people beyond his base, as demonstrated by his comparison to Saddam Hussein's Iraq, is a major problem for the United States, as it requires the president to build bridges and form alliances in order to get things done.
The article discussed the chaotic scenes that occurred prior to and after the Euro 2016 match between France and England in Marseille, involving hundreds of fans, objects being thrown, and police firing tear gas, as well as the violent rivalry between the two teams' supporters.
The article discussed the impact of a leaked video on two Democratic operatives, Robert Creamer and Scott Foval, who lost their jobs due to the investigations by James O'Keefe's Project Veritas Action, and the involvement of super-PACs in the unified Democratic campaign for Hillary Clinton.
The article discussed the surrender of Japan to the Allied forces after the atomic bombing of Nagasaki, and the joyous celebration of the American peopl

In [69]:
def strip_sentence(sentence):
    if sentence.startswith('The article discussed how'):
        stripped_sentence = sentence.replace('The article discussed how', '').strip()
    elif sentence.startswith('The article discussed'):
        stripped_sentence = sentence.replace('The article discussed', '').strip()
    return stripped_sentence

'the corruption scandals of Hillary Clinton, including her appointment of a commodities trader to a Top Secret nuclear weapons security advisory board, which was linked to his donations to the Clinton Foundation, and the emails acquired by Citizens United through the Freedom of Information Act which showed that career Department officials questioned the appointment.'

In [70]:
def extract_events(sentence):
    prompt = """
    An event graph describes an event in graph structure.
    It should contain one or more 'participants', which are the major participants in the event,
    and a 'trigger', which is a verb that describes what happens between the participants.
    Reorganize the sentence below into one or more event graph.
    The triggers and participants should be human-readable.
    Reply with each line being an event graph in the format:
    [trigger1], [participant 1], [participant 2], ...
    [trigger2], [participant 1], [participant 2], ...
    Sentence: {sentence}
    """.format(sentence=sentence)
    events = request_chatgpt(prompt)
    return events


Appointed, Hillary Clinton, Commodities Trader, Top Secret Nuclear Weapons Security Advisory Board
Acquired, Citizens United, Emails, Freedom of Information Act
Questioned, Career Department Officials, Appointment


'Appointed, Hillary Clinton, Commodities Trader, Top Secret Nuclear Weapons Security Advisory Board\nAcquired, Citizens United, Emails, Freedom of Information Act\nQuestioned, Career Department Officials, Appointment'